In [ ]:
from datetime import datetime
from glob import glob
from os import environ, path
from pickle import dump

from boto3 import client
from face_recognition import face_encodings, load_image_file

### Initialize config parameters

In [ ]:
images_folder = './face-images/'
output_file_path = environ.get('MODEL_FILENAME', 'model.data')
timestamp = datetime.now().strftime('%y%m%d%H%M')

s3_access_key = environ.get('AWS_ACCESS_KEY_ID')
s3_secret_key = environ.get('AWS_SECRET_ACCESS_KEY')
s3_endpoint_url = environ.get('AWS_S3_ENDPOINT')
s3_bucket_name = environ.get('AWS_S3_BUCKET')

print(f'output file path: {output_file_path}')
print(f'time stamp: {timestamp}')
print(f'S3 access key: {s3_access_key}')
print(f'S3 secret key: {s3_secret_key}')
print(f'S3 endpoint URL: {s3_endpoint_url}')
print(f'S3 bucket name: {s3_bucket_name}')

### Read face image files

In [ ]:
image_file_paths = glob(path.join(images_folder, "*.jpg"))
image_names = [
    file_path.split('/')[-1].rstrip('.jpg')
    for file_path in image_file_paths
]
print(image_file_paths)
print(image_names)

### Encode faces

In [ ]:
def encode_face(image_file_path):
    face_image = load_image_file(image_file_path)
    encoding = face_encodings(face_image, num_jitters=100, model='large')[0]
    return encoding

encodings = [
    encode_face(image_file_path)
    for image_file_path in image_file_paths
]

### Serialize encodings and metadata

In [ ]:
model_package = (encodings, image_names)

with open(output_file_path, 'wb') as outputfile:
    dump(model_package, outputfile)
    print(f'Dumped model package to {output_file_path}.')

### Upload encodings to S3

In [ ]:
s3_object_name = f'{timestamp}/{output_file_path}'

s3_client = client(
    's3', endpoint_url=s3_endpoint_url,
    aws_access_key_id=s3_access_key, aws_secret_access_key=s3_secret_key
)
with open(output_file_path, 'rb') as model_package_file:
    s3_client.upload_fileobj(model_package_file, s3_bucket_name, s3_object_name)

    print(f'Uploaded {s3_object_name} to bucket {s3_bucket_name} '
          f'to S3 storage at {s3_endpoint_url}')

### Test serialized model format

In [ ]:
from pickle import load

with open(output_file_path, 'rb') as inputfile:
    loaded_models = load(inputfile)
    
print(loaded_models)